In [1]:
import os
import numpy as np
import cv2 as cv
from utils import Utils
from sklearn.mixture import GaussianMixture
import time

In [2]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [15]:
# Load the image
img_width = 480
start_time = time.time()
img = cv.imread('../Dataset_0-5/men/4/4_men (1).JPG')
h, w = img.shape[:2]
new_height = int(h * img_width / w)
img_size = (img_width, new_height)
img = cv.resize(img, img_size)    # resize image

# Convert the image from BGR to YCbCr
ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

# Extract Cr and Cb channels
cr = ycbcr_image[:, :, 1]
cb = ycbcr_image[:, :, 2]

# Reshape channels into a 1D array
cr_flat = cr.flatten().reshape(-1, 1)
cb_flat = cb.flatten().reshape(-1, 1)

# Concatenate channels into a single array
x = np.concatenate((cr_flat, cb_flat), axis=1)

# Fit Gaussian mixture model
gmm = GaussianMixture(n_components=2)
gmm.fit(x)

# Predict probabilities of each pixel belonging to background or foreground
probs = gmm.predict_proba(x)
probs_bg = probs[:, 0].reshape(cr.shape)
probs_fg = probs[:, 1].reshape(cr.shape)

# Threshold probabilities to extract foreground object from background
thresh = 0.5
mask1 = (probs_fg > thresh).astype(np.uint8) * 255
# Threshold probabilities to extract background object from foreground
# mask2 = (probs_bg > thresh).astype(np.uint8) * 255
# Apply mask to original image
result1 = cv.bitwise_and(img, img, mask=mask1)
result2 = cv.bitwise_and(img, img, mask=~(mask1))
black_pixels1 = (result1 == np.array([0, 0, 0])).all(axis=2)
black_pixels2 = (result2 == np.array([0, 0, 0])).all(axis=2)

# extract the hand from the image
out = cv.bitwise_or(result1, result2)
end_time = time.time()
totTime = end_time - start_time

In [52]:
for i in range(1, 20):
    img = cv.imread(f'../Dataset_0-5/men/2/2_men ({i}).JPG')
    img_width = 120
    h, w = img.shape[:2]
    new_height = int(h * img_width / w)
    img_size = (img_width, new_height)
    img = cv.resize(img, img_size)    # resize image

    # Convert the image from BGR to YCbCr
    ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

    # Extract Cr and Cb channels
    cr = ycbcr_image[:, :, 1]
    cb = ycbcr_image[:, :, 2]

    # Reshape channels into a 1D array
    cr_flat = cr.flatten().reshape(-1, 1)
    cb_flat = cb.flatten().reshape(-1, 1)

    # Concatenate channels into a single array
    x = np.concatenate((cr_flat, cb_flat), axis=1)

    # Fit Gaussian mixture model
    gmm = GaussianMixture(n_components=3)
    gmm.fit(x)

    # Predict probabilities of each pixel belonging to background or foreground
    probs = gmm.predict_proba(x)
    probs_bg = probs[:, 0].reshape(cr.shape)
    probs_fg = probs[:, 1].reshape(cr.shape)
    probs_sh = probs[:, 2].reshape(cr.shape)
    # Threshold probabilities to extract foreground object from background
    thresh = 0.5
    mask1 = (probs_fg > thresh).astype(np.uint8) * 255
    mask2 = (probs_bg > thresh).astype(np.uint8) * 255
    mask3 = (probs_sh > thresh).astype(np.uint8) * 255
    # Apply mask to original image
    result1 = cv.bitwise_and(img, img, mask=mask1)
    result2 = cv.bitwise_and(img, img, mask=(mask2))
    result3 = cv.bitwise_and(img, img, mask=(mask3))
# extract the hand from the image
# Convert mask to grayscale and apply thresholding to extract hand from background
# gray_mask = cv.cvtColor(mask, cv.COLOR_GRAY2BGR)
# _, binary_mask = cv.threshold(gray_mask, 0, 255, cv.THRESH_BINARY)

# # Apply binary mask to original image to extract hand from background
# hand = cv.bitwise_and(img, img, mask=binary_mask)
# Convert image to YCrCb color space
# img_ycrcb1 = cv.cvtColor(result1, cv.COLOR_BGR2YCrCb)
# img_ycrcb2 = cv.cvtColor(result2, cv.COLOR_BGR2YCrCb)

# # Define range of skin color in YCrCb color space
# lower_skin = np.array([0, 135, 85], dtype=np.uint8)
# upper_skin = np.array([255, 180, 135], dtype=np.uint8)

# # Extract skin color from image
# mask1 = cv.inRange(img_ycrcb1, lower_skin, upper_skin)
# mask2 = cv.inRange(img_ycrcb2, lower_skin, upper_skin)

# # Apply morphological operations to remove noise from mask
# kernel = np.ones((3, 3), np.uint8)
# mask1 = cv.erode(mask1, kernel, iterations=1)
# mask1 = cv.dilate(mask1, kernel, iterations=1)

# mask2 = cv.erode(mask2, kernel, iterations=1)
# mask2 = cv.dilate(mask2, kernel, iterations=1)

# # Apply mask to original image to extract hand
# result1 = cv.bitwise_and(img, img, mask=mask1)
# result2 = cv.bitwise_and(img, img, mask=mask2)
# resGrey1 = Utils.getMaskedHand(result1)
# resGrey2 = Utils.getMaskedHand(result2)
# mean1 = np.mean(resGrey1)
# mean2 = np.mean(resGrey2)
# print(mean1, mean2)
# if mean1 > mean2:
#     result = result1
# else:
#     result = result2
cv.imshow('result1',  cv.resize(result1, (1024, 512)))
cv.waitKey(0)
cv.imshow('result2',  cv.resize(result2, (1024, 512)))
cv.waitKey(0)
cv.imshow('result3',  cv.resize(result3, (1024, 512)))
cv.waitKey(0)
# # while True:
# class_dir = os.path.join(menPath, f"{0}")
# imgPath = os.path.join(class_dir, f'{0}_men ({1}).JPG')
# img = cv.imread(imgPath)
# img = getThresholdedHand(img)
# cv.imshow("img", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

KeyboardInterrupt: 

In [18]:
for i in range(1, 21):
    out = Utils.extract_hand( cv.imread(f"../Dataset_0-5/men/0/0_men ({i}).JPG"))
    cv.imshow(f"out{i}",  cv.resize(out, (1024, 512)))
    cv.waitKey(0)
    cv.destroyAllWindows()

Time:  1.5877845287322998
Time:  1.8996868133544922


KeyboardInterrupt: 

In [ ]:
# Load the image
img = cv.imread('../Dataset_0-5/men/4/4_men (1).JPG')

# Convert the image from BGR to YCbCr
ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

# Extract the Cb and Cr channels from the YCbCr image
cb_channel = ycbcr_image[:, :, 1]
cr_channel = ycbcr_image[:, :, 2]

# Reshape the Cb and Cr channels into 1D arrays
cb_channel_1d = np.reshape(cb_channel, (-1, 1))
cr_channel_1d = np.reshape(cr_channel, (-1, 1))

# Concatenate the Cb and Cr channels into a single 2D array
cbcr_channels = np.concatenate((cb_channel_1d, cr_channel_1d), axis=1)

# Fit a Gaussian mixture model to the Cb and Cr channels
gmm = GaussianMixture(n_components=2)
gmm.fit(cbcr_channels)

# Predict the labels for each pixel in the Cb and Cr channels
labels = gmm.predict(cbcr_channels)

# Reshape the labels back into a 2D array
labels_2d = np.reshape(labels, cb_channel.shape)

# Convert the labels to uint8 data type
labels_2d_uint8 = labels_2d.astype(np.float64)
print(labels_2d_uint8.max())
# Display the labels as an image
cv.imshow('Labels',  cv.resize(labels_2d_uint8, (1024, 512)))
cv.waitKey(0)


In [43]:
# Load image
img = cv.imread('../Dataset_0-5/men/2/2_men (9).JPG')

# Convert image to YCrCb color space
img_ycrcb = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)

# Define range of skin color in YCrCb color space
lower_skin = np.array([0, 135, 85], dtype=np.uint8)
upper_skin = np.array([255, 180, 135], dtype=np.uint8)

# Extract skin color from image
mask = cv.inRange(img_ycrcb, lower_skin, upper_skin)

# Apply morphological operations to remove noise from mask
kernel = np.ones((3, 3), np.uint8)
mask = cv.erode(mask, kernel, iterations=1)
mask = cv.dilate(mask, kernel, iterations=1)

# Apply mask to original image to extract hand
result = cv.bitwise_and(img, img, mask=mask)

cv.imshow('r',  cv.resize(result, (1024, 512)))
cv.waitKey(0)

-1